This notebook calculates the alignment factors $\alpha$ and the probability in the face-on degenerate region of the $\cos \iota$ posterior pdf, for isotropic sources in various networks

### Setup and imports

Import the relevant codes, set the plotting style

In [1]:
%matplotlib inline
import sys
from numpy import *
import numpy as np
import pylab as plt
from simple_pe import localize
import lal
import logging

import scipy.integrate as integrate
from scipy.integrate import quad

import astropy.units as u


sys.path.append('')
sys.path.append('simple_pe')

import fstat
import likelihood
from scipy.stats import norm
import detectors

import h5py

import antenna_beam_pattern
from pycbc.detector import Detector

/cvmfs/oasis.opensciencegrid.org/ligo/sw/pycbc/x86_64_rhel_7/virtualenv/pycbc-v1.9.4/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = {}
posterior = {}
events = ["GW150914", "LVT151012", "GW151226", "GW170104", "GW170608", "GW170729", "GW170809", "GW170814", "GW170817", "GW170818", "GW170823"]

for event in events:
    if (event == "GW150914" or event == "LVT151012") or event == "GW151226":
        data[event] = h5py.File("posterior_data/%s_data.h5" %event)
        print "%s_data.h5" %event
        posterior[event] = data[event]["overall_post"]
    
    else:
        posterior[event] = genfromtxt('posterior_data/%s_data.dat' %event,names=True)
        print "%s_data.dat" %event

GW150914_data.h5
LVT151012_data.h5
GW151226_data.h5
GW170104_data.dat
GW170608_data.dat
GW170729_data.dat
GW170809_data.dat
GW170814_data.dat
GW170817_data.dat
GW170818_data.dat
GW170823_data.dat


Set the plotting parameters:

In [3]:
plt.rcParams.update({
    "lines.markersize": 6,
    "lines.markeredgewidth": 1.5,
    "lines.linewidth": 1.0,
    "font.size": 20,
    "axes.titlesize": 20,
    "axes.labelsize": 20,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "legend.fontsize": 20,
})

Set the parameters

# Inject uniform in volume

In [4]:
from simple_pe import cosmology
from simple_pe import merger_rate_evolution
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.interpolate import interp1d
import astropy.units as u

In [5]:
# inject_uniform_in_componant_masses = False
sfr_injection = False #do you want the rate to follow the star formation rate (with a 
                     #1/t delay distribution between formation and merger and a minimum 
                     #delay of 20 Myr)
z_max_star_formation = 20 # the maximum z at which it is assumed stars can form

In [6]:
generation = '2G'

In [7]:
ignore_cosmology = True
zmax = cosmology.red_dl(440)
ntrials = 46848
npts_injection = 50l

## Calculate distribution of events in volume 

In [8]:
R0=1500 * u.Gpc**-3 * u.yr**-1 # set rate to BNS rate

if sfr_injection: # then the local merger rate follows sfr
    
    def integrand(z, R0=R0):
        return merger_rate_evolution.integrand(z,R0, z_max = z_max_star_formation)

else:          # otherwise the local merger rate is constant
   
    def integrand(z): # eq C3 in BBH merger paper (modified since we don't include the prob of detection, and inserting solid angle)
        dv_dz=cosmology.cosmo.differential_comoving_volume(z).to(u.Gpc**3 / u.sr)
        return 4*np.pi *u.sr * dv_dz * R0 /(1+z) * u.yr

Make an interpolating function for the redshift distribution

In [9]:
dc_cum = {}
inj = {}
for event in events:
    z = linspace(0, zmax, npts_injection) #used to have 1000 points for old injections
    if sfr_injection: inj[event] = array([integrand(zi) for zi in z])
    else: inj[event] = integrand(z) 
    cp = cumsum(inj[event])/sum(inj[event])
    dc_cum[event] = interp1d(cp, cosmology.cosmo.comoving_distance(z))

In [10]:
Dcos = {}
Zs = {}
for event in events[:1]:
    dcos = []
    zs = []
    for i in xrange(100000):
        dco = dc_cum[event](random.uniform())
        dcos.append(dco)
        zs.append(cosmology.z_interp(dco))
    Dcos[event] = dcos
    Zs[event] = zs

## Generate events

In [12]:
all_lists = {}
alpha_lists = {}
cosi_lists = {}
num_found = {}
P_fo_lists = {}
all_timedelays = {}
H1_F_plus = {}
L1_F_plus = {}
H1_F_cross = {}
L1_F_cross = {}

det_h1 = Detector('H1')
det_l1 = Detector('L1')
det_v1 = Detector('V1')

for event in events:
    n = localize.network()
    n.set_configuration("HL", seed=0) # set the seed so all the injections are consistent
    all_list = []
    alpha_list = []
    cosi_list = []
    P_fo_list = []
    nf = 0
    timedelays = []
    H1_F_plus_list = []
    L1_F_plus_list = []
    H1_F_cross_list = []
    L1_F_cross_list = []
    
    for i in xrange(len(posterior[event]["time"])):
        if i%1000 == 0:
            print 'event number %s out of %s for %s' % (i,len(posterior[event]["time"]),event)
        
        x = localize.event(n, comoving_distn=dc_cum[event])
        x.gps = lal.LIGOTimeGPS(int(np.modf(posterior[event]["time"][i-1])[1]),int(np.modf(posterior[event]["time"][i-1])[0]*1000000000))
        x.gmst = lal.GreenwichMeanSiderealTime(x.gps)
        if (event == "GW150914" or event == "LVT151012") or event == "GW151226":
            x.ra = posterior[event]["right_ascension"][i-1]
            x.dec = posterior[event]["declination"][i-1]
        else:
            x.ra = posterior[event]["ra"][i-1]
            x.dec = posterior[event]["dec"][i-1]
        n.set_event_config(x, ignore_cosmology = ignore_cosmology)
        x.add_network(n)
        
        all_list.append(x)
        if x.detected:
            nf += 1
            x.orientate()
            alpha_list.append(x.alpha)
            cosi_list.append(x.cosi)
            P_fo_list.append(x.P_fo)
            timedelays.append(lal.ArrivalTimeDiff(detectors.detectors("H1")[0]["H1"], detectors.detectors("L1")[0]["L1"], x.ra, x.dec, x.gps))
            h1_fplus, h1_fcross = det_h1.antenna_pattern(x.ra, x.dec, x.psi, x.gmst)
            l1_fplus, l1_fcross = det_l1.antenna_pattern(x.ra, x.dec, x.psi, x.gmst)
            H1_F_plus_list.append(h1_fplus)
            L1_F_plus_list.append(l1_fplus)
            H1_F_cross_list.append(h1_fcross)
            L1_F_cross_list.append(l1_fcross)
            
    num_found[event] = nf
        
    all_list=np.array(all_list)
    cosi_list=array(cosi_list)
    alpha_list=array(alpha_list)
    P_fo_list = array(P_fo_list)
    all_lists[event] = all_list
    cosi_lists[event] = cosi_list
    alpha_lists[event] = alpha_list
    P_fo_lists[event] = P_fo_list
    all_timedelays[event] = timedelays
    H1_F_plus[event] = H1_F_plus_list
    L1_F_plus[event] = L1_F_plus_list
    H1_F_cross[event] = H1_F_cross_list
    L1_F_cross[event] = L1_F_cross_list

event number 0 out of 46848 for GW150914
event number 1000 out of 46848 for GW150914
event number 2000 out of 46848 for GW150914
event number 3000 out of 46848 for GW150914
event number 4000 out of 46848 for GW150914
event number 5000 out of 46848 for GW150914
event number 6000 out of 46848 for GW150914
event number 7000 out of 46848 for GW150914
event number 8000 out of 46848 for GW150914
event number 9000 out of 46848 for GW150914
event number 10000 out of 46848 for GW150914
event number 11000 out of 46848 for GW150914
event number 12000 out of 46848 for GW150914
event number 13000 out of 46848 for GW150914
event number 14000 out of 46848 for GW150914
event number 15000 out of 46848 for GW150914
event number 16000 out of 46848 for GW150914
event number 17000 out of 46848 for GW150914
event number 18000 out of 46848 for GW150914
event number 19000 out of 46848 for GW150914
event number 20000 out of 46848 for GW150914
event number 21000 out of 46848 for GW150914
event number 22000 out 

## Save the data

Save the data in a text file to be presented in the Fraction_face_on notebook in this directory

In [13]:
for event in events:
    savetxt('network_data/%s_cosi_P_face-on.txt' % event, array([cosi_lists[event], P_fo_lists[event], all_timedelays[event], H1_F_plus[event], H1_F_cross[event], L1_F_plus[event], L1_F_cross[event]]))
    savetxt('network_data/%s_cosi_P_face-on_alpha.txt' % event, array([cosi_lists[event], P_fo_lists[event], alpha_lists[event], all_timedelays[event], H1_F_plus[event], H1_F_cross[event], L1_F_plus[event], L1_F_cross[event]]))